In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import os
import gc
import random
from collections import defaultdict, Counter
from typing import List, Dict
import joblib
import pickle
from datetime import datetime

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [4]:
%env DATA_ROOT /content/data

env: DATA_ROOT=/content/data


In [5]:
!mkdir -p ${DATA_ROOT} && \
    cp /gdrive/MyDrive/amazon_kdd_2023/data/external/all_csv_files.zip ${DATA_ROOT} && \
    cd ${DATA_ROOT} && \
    unzip -n all_csv_files.zip

Archive:  all_csv_files.zip
  inflating: all_csv_files.csv       


In [9]:
rating = pl.read_csv(
    "/content/data/all_csv_files.csv",
    has_header=False,
    new_columns=["product_id", "reviewer_id", "rate", "timestamp"],
)

In [10]:
rating.head()

product_id,reviewer_id,rate,timestamp
str,str,f64,i64
"""B00ZGTMHVU""","""A1HGH7NF01K3W7…",5.0,1451001600
"""B013CG1EVC""","""A3GDQGSD4C0HOB…",5.0,1496966400
"""B013CG1EVC""","""A1C88HIIEVN0QN…",5.0,1445904000
"""B013CG1EVC""","""A1G5MTDE3A5LSL…",5.0,1440892800
"""B013CG1XS6""","""A3SOHSDN51M5NQ…",5.0,1441497600


In [11]:
rating.describe()

describe,product_id,reviewer_id,rate,timestamp
str,str,str,f64,f64
"""count""","""233055327""","""233055327""",2.33055327e8,2.33055327e8
"""null_count""","""0""","""0""",0.0,0.0
"""mean""",null,null,4.23219,1.4290e9
"""std""",null,null,1.246512,8.5777e7
"""min""","""0000000116""","""A0000040I1OM9N…",0.0,8.325504e8
"""max""","""B01HJI17Y8""","""AZZZZXVAOWWME""",5.0,1.5389e9
"""median""",null,null,5.0,1.4463e9
"""25%""",null,null,4.0,1.4029e9
"""75%""",null,null,5.0,1.4831e9


In [26]:
rating["rate"].value_counts()

rate,counts
f64,u32
0.0,9
2.0,10979213
1.0,18256432
5.0,148649950
3.0,17809027
4.0,37360696


In [12]:
rating["product_id"].n_unique()

15167257

In [18]:
rating[["product_id", "rate"]].groupby("product_id").count().mean()

product_id,count
str,f64
null,15.365687


In [19]:
rating[["product_id", "rate"]].groupby("product_id").count().max()

product_id,count
str,u32
"""B01HJI17Y8""",58150


In [20]:
rating[["product_id", "rate"]].groupby("product_id").count().min()

product_id,count
str,u32
"""0000000116""",1


In [25]:
ts_max = rating["timestamp"].max()
ts_min = rating["timestamp"].min()
dt_max = datetime.fromtimestamp(ts_max)
dt_min = datetime.fromtimestamp(ts_min)
print(dt_max)
print(dt_min)

2018-10-07 00:00:00
1996-05-20 00:00:00


In [29]:
rating = rating.with_columns(pl.col("rate").cast(pl.Int8))

In [30]:
rating["rate"].value_counts()

rate,counts
i8,u32
0,9
1,18256432
2,10979213
3,17809027
4,37360696
5,148649950


In [31]:
rating[["product_id", "rate"]].write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/common/rating_01.parquet")